In [1]:
import math
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 2500)

# Loading historical data
hs_param = pd.read_csv('/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Historical VaR/Data/historical parameters.csv')
# Setting up the parameters differently
decay_factor = hs_param.decay_factor.iloc[0]
holding_period = hs_param.holding_period.iloc[0]
# print(holding_period)
confidence_level = hs_param.confidence_level.iloc[0]

rates_E = pd.read_csv('/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Historical VaR/Data/rates_instrument.csv')
rates_E = rates_E[rates_E["instrument_type"] == 'E']
rates_E.instrument = rates_E.instrument.astype('str')
rates_E["key"] = "instru" + '_'+rates_E.instrument

# Simulated Instrument rates
simulated_instruments = pd.DataFrame([])

for a, h in rates_E.groupby('instrument'):
    g = h.filter(["instrument", "instrument_type", "value", "key"])
    new_row = g.iloc[0].to_dict()
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    g = g.reset_index(drop = True)
    g['simulated_rates'] = np.log(g["value"]/g["value"].shift(-holding_period, fill_value = g["value"][0]))
    
    g = pd.concat([top_row, g]).reset_index(drop = True)

    g['new_value'] = g['simulated_rates'].apply(lambda x: np.exp(x)*g["value"].iloc[0])
    simulated_instruments = simulated_instruments.append(g)



rates_exchange = pd.read_csv("/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Historical VaR/Data/rates_exchange.csv")

if len(rates_exchange) != 0:
    rates_exchange["key"] = rates_exchange["currency_from"] + "_"+rates_exchange["currency_to"]

simulated_exchange = pd.DataFrame([])
for i,k in rates_exchange.groupby('key'):
    j = k.filter(["currency_from", "currency_to", "rate", "key"])
    new_row = j.iloc[0].to_dict()
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    
    j = j.reset_index(drop = True)
    j['simulated_rates'] = np.log(j['rate']/j['rate'].shift(-holding_period, fill_value = j["rate"][0]))
    j = pd.concat([top_row, j]).reset_index(drop = True)
    j["new_rate"] = j['simulated_rates'].apply(lambda x: np.exp(x)*j["rate"].iloc[0])
    simulated_exchange = simulated_exchange.append(j)

simulated_exchange.head(5)

tx_eq = pd.read_csv('/mnt/c/Users/shraddha.talekar/Documents/market_mix_modelling/VaR/Historical VaR/Data/transaction_equity.csv')
n=hs_param['no_of_days'].iloc[0]

def get_mtm_bc(idx,tx_df, ex_df, instr_df):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        h["keyMatch"] = h['currency'] + '_' + h['currency_base']
        keyMatch = h["keyMatch"].iloc[0]
        instrMatch = h["instrument"].iloc[0]
        unitCount = h['unit_count'].iloc[0]
        
        crss_rate_df = ex_df[ex_df['key']==keyMatch]
        cross_rate = crss_rate_df['new_rate'].iloc[0]
        
        equity_rate_df = instr_df[instr_df['instrument'] == instrMatch]
        equity_rate = equity_rate_df['new_value'].iloc[0]
        
        mtm_bc = cross_rate * equity_rate * unitCount
        result = {'idx': idx, "env":k,"instrument": instrMatch, "unit_count":unitCount, "accounting_book":h["accounting_book"].iloc[0],
                 "accounting_portfolio": h["accounting_portfolio"].iloc[0], "currency":h["currency"].iloc[0], "mtm_bc":mtm_bc}
        res_df.append(result)

    return res_df

final_df = pd.DataFrame()
for i in range(0, n):
    exch_rates = simulated_exchange[simulated_exchange.index == i]
    instr_rates = simulated_instruments[simulated_instruments.index == i]
    result = get_mtm_bc(i, tx_eq, exch_rates, instr_rates)
    res = pd.DataFrame(result)

    final_df = final_df.append(res)



final_df.to_csv("final_result.csv",index = False)
simulated_exchange.to_csv("simulated_exchange.csv", index=False)
simulated_instruments.to_csv("simulated_instrument.csv", index = False)

/tmp/ipykernel_214/3217483587.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_214/3217483587.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_214/3217483587.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_214/3217483587.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_214/3217483587.py:34: FutureWarning: The frame.append method is deprecate

In [2]:
final_df

,idx,env,instrument,unit_count,accounting_book,accounting_portfolio,currency,mtm_bc
0,0,HDFCbank1942,HDFCbank,11000,1,1,INR,1.501555e+07
1,0,Hindustanlever914,HLL,7500,1,1,INR,1.953375e+07
2,0,Infosys203,Infosys,13550,1,1,INR,2.053232e+07
3,0,Reliance1908,Reliance,6000,1,1,INR,1.501800e+07
4,0,TCS108,TCS,10000,1,1,INR,3.164800e+07
0,1,HDFCbank1942,HDFCbank,11000,1,1,INR,1.494766e+07
1,1,Hindustanlever914,HLL,7500,1,1,INR,2.036329e+07
2,1,Infosys203,Infosys,13550,1,1,INR,2.085785e+07
3,1,Reliance1908,Reliance,6000,1,1,INR,1.558557e+07
4,1,TCS108,TCS,10000,1,1,INR,3.071673e+07


In [3]:
#df.sort_values("city08")
final_df = final_df.sort_values(by =["env","idx"], ascending = [True, True])
final_df

,idx,env,instrument,unit_count,accounting_book,accounting_portfolio,currency,mtm_bc
0,0,HDFCbank1942,HDFCbank,11000,1,1,INR,1.501555e+07
0,1,HDFCbank1942,HDFCbank,11000,1,1,INR,1.494766e+07
0,2,HDFCbank1942,HDFCbank,11000,1,1,INR,1.496504e+07
0,3,HDFCbank1942,HDFCbank,11000,1,1,INR,1.492583e+07
0,4,HDFCbank1942,HDFCbank,11000,1,1,INR,1.510761e+07
0,5,HDFCbank1942,HDFCbank,11000,1,1,INR,1.504952e+07
0,6,HDFCbank1942,HDFCbank,11000,1,1,INR,1.517085e+07
0,7,HDFCbank1942,HDFCbank,11000,1,1,INR,1.552763e+07
0,8,HDFCbank1942,HDFCbank,11000,1,1,INR,1.556801e+07
0,9,HDFCbank1942,HDFCbank,11000,1,1,INR,1.549581e+07


In [4]:
#change = final_df['mtm_bc'].iloc[0]

In [5]:
def change(final_df):
    
    change_df = pd.DataFrame()
    
    for i,j in final_df.groupby('instrument'):
        curr_val = j['mtm_bc'].iloc[0]
        j['change_in_mtm'] = j['mtm_bc'].apply(lambda x: x - curr_val)
        
        change_df = change_df.append(j)
        
    return change_df
        

In [6]:
change_df= change(final_df)

/tmp/ipykernel_214/1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_214/1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_214/1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_214/1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = change_df.append(j)
/tmp/ipykernel_214/1292147285.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  change_df = chang

In [7]:
change_df

,idx,env,instrument,unit_count,accounting_book,accounting_portfolio,currency,mtm_bc,change_in_mtm
0,0,HDFCbank1942,HDFCbank,11000,1,1,INR,1.501555e+07,0.000000e+00
0,1,HDFCbank1942,HDFCbank,11000,1,1,INR,1.494766e+07,-6.789164e+04
0,2,HDFCbank1942,HDFCbank,11000,1,1,INR,1.496504e+07,-5.051068e+04
0,3,HDFCbank1942,HDFCbank,11000,1,1,INR,1.492583e+07,-8.971781e+04
0,4,HDFCbank1942,HDFCbank,11000,1,1,INR,1.510761e+07,9.206210e+04
0,5,HDFCbank1942,HDFCbank,11000,1,1,INR,1.504952e+07,3.397435e+04
0,6,HDFCbank1942,HDFCbank,11000,1,1,INR,1.517085e+07,1.553005e+05
0,7,HDFCbank1942,HDFCbank,11000,1,1,INR,1.552763e+07,5.120839e+05
0,8,HDFCbank1942,HDFCbank,11000,1,1,INR,1.556801e+07,5.524615e+05
0,9,HDFCbank1942,HDFCbank,11000,1,1,INR,1.549581e+07,4.802580e+05


#### Total change in mtm

In [8]:
total_change_in_mtm = change_df.groupby('idx')[['change_in_mtm']].sum()

In [9]:
total_change_in_mtm = total_change_in_mtm['change_in_mtm'].tolist()

### Decay Factor

In [10]:
if (decay_factor == 0):
    VaR_exact_equity_hp = -(np.quantile(total_change_in_mtm, 0.01))
    total_change_in_mtm_sorted = sorted(total_change_in_mtm, reverse = True)
    nearby = np.floor(confidence_level * len(total_change_in_mtm))
    nearby = int(nearby)
    nearby_nxt = nearby + 1
    
    diff1 = pd.to_numeric(VaR_exact_equity_hp + total_change_in_mtm_sorted[nearby])
    diff2 = pd.to_numeric(- total_change_in_mtm_sorted[nearby_nxt] - VaR_exact_equity_hp)
    
    if (diff1 == diff2):
        VaR_nearby_equity_hp = - total_change_in_mtm_sorted[nearby + 1]
    elif (diff1 < diff2):
        VaR_nearby_equity_hp = -total_change_in_mtm_sorted[nearby]
    else:
        VaR_nearby_equity_hp = - total_change_in_mtm_sorted[nearby + 1]
        
else:
    decay_weights = (1-decay_factor)* ((decay_factor ** range(0,(len(change_df['idx'].unique())-1)))/((1 - decay_factor) ** (len(change_df['idx'].unique() -1))))
    total_mtm = change_df.groupby('idx')[['mtm_bc']].sum
    mtm_returns = []

    for i in range(1,(len(change_df['idx'].unique() -1))):
                   fv = total_mtm['mtm_bc'][0]
                   returns = (total_mtm['mtm_bc'][i] - fv)/fv
                   mtm_returns.append(returns) 
                    
    decay_weights = pd.DataFrame(decay_weights, columns = ['decay_weights'])
    mtm_returns = pd.DataFrame(mtm_returns, columns = ['mtm_returns'])
    mtm = pd.concat([mtm_returns,decay_weights], axis = 1)
    mtm = mtm.sort_values(['mtm_returns'])
    mtm['cum_decay_weights'] = np.cumsum(mtm['decay_weights'])
    a = 1-confidence_level
    mtm1 = mtm.loc[mtm['cum_decay_weights'] < a]
                                         
    if (len(mtm1)==0):
                  VaR_exact_equity_hp = np.abs(- mtm['mtm_returns'][0] * total_mtm['mtm_bc'][0])

                  VaR_nearby_equity_hp = VaR_exact_equity_hp

    else:
                  index = len(mtm1['cum_decay_weights'])
                                         
                  interpolated_return_exact = mtm['mtm_returns'][index] + ((mtm['mtm_returns'][index + 1] - mtm['mtm_returns'][index]) * ((1 - confidence_level) - mtm['cum_decay_weights'][index]) / (mtm['cum_decay_weights'][index + 1] - mtm['cum_decay_weights'][index] ))
    
                  VaR_exact_equity_hp =  ((total_mtm[1] * (interpolated_return_exact)))
    
                  if((((1 - confidence_level) - mtm['cum_decay_weights'][index]) < (mtm['cum_decay_weights'][index + 1] - (1 - confidence_level) ))):
            
                            interpolated_return_nearby = mtm['mtm_returns'][index]
                  else:
                            index = len(mtm1['cum_decay_weights'])
                            interpolated_return_nearby = mtm['mtm_returns'][index + 1]
    
                            VaR_nearby_equity_hp = np.abs(total_mtm['mtm_bc'][0] * (interpolated_return_nearby))
                                         
                                         
    
    

In [11]:
VaR_exact_equity_hp

7405630.109225428

In [12]:
VaR_nearby_equity_hp

7489518.753564823